In [1]:
# Taken from https://medium.com/data-and-beyond/complete-guide-to-building-bert-model-from-sratch-3e6562228891
import os
from pathlib import Path
import torch
import re
import random
from tokenizers import BertWordPieceTokenizer
import tqdm
from torch.utils.data import Dataset, DataLoader
import itertools
import math
import torch.nn.functional as F
import numpy as np
from torch.optim import Adam
import datasets
import transformers
from transformers import BertTokenizer

In [2]:
MAX_LEN = 64

### loading all data into memory
corpus_movie_conv = './datasets/movie_conversations.txt'
corpus_movie_lines = './datasets/movie_lines.txt'
with open(corpus_movie_conv, 'r', encoding='iso-8859-1') as c:
    conv = c.readlines() 
with open(corpus_movie_lines, 'r', encoding='iso-8859-1') as l:
    lines = l.readlines()

In [3]:
### splitting text using special lines
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]
list(lines_dic.items())[0:4]

[('L1045', 'They do not!\n'),
 ('L1044', 'They do to!\n'),
 ('L985', 'I hope so.\n'),
 ('L984', 'She okay?\n')]

In [4]:
### generate question answer pairs
pairs = []
for con in conv:
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i == len(ids) - 1:
            break

        first = lines_dic[ids[i]].strip()  
        second = lines_dic[ids[i+1]].strip() 

        qa_pairs.append(' '.join(first.split()[:MAX_LEN]))
        qa_pairs.append(' '.join(second.split()[:MAX_LEN]))
        pairs.append(qa_pairs)
print(len(pairs))
print(pairs[0:3])

221616
[['Can we make this quick? Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad. Again.', "Well, I thought we'd start with pronunciation, if that's okay with you."], ["Well, I thought we'd start with pronunciation, if that's okay with you.", 'Not the hacking and gagging and spitting part. Please.'], ['Not the hacking and gagging and spitting part. Please.', "Okay... then how 'bout we try out some French cuisine. Saturday? Night?"]]


In [5]:
### save data as txt file
os.mkdir('./data')
text_data = []
file_count = 0

for sample in tqdm.tqdm([x[0] for x in pairs]):
    text_data.append(sample)

    # once we hit the 10K mark, save to file
    if len(text_data) == 10000:
        with open(f'./data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

paths = [str(x) for x in Path('./data').glob('**/*.txt')]

100%|████████████████████████████████████████████████████████████████████████| 221616/221616 [00:00<00:00, 2307967.99it/s]


In [6]:
### training own tokenizer
tokenizer = BertWordPieceTokenizer(
    clean_text=True,
    handle_chinese_chars=False,
    strip_accents=False,
    lowercase=True
)

tokenizer.train( 
    files=paths,
    vocab_size=30_000, 
    min_frequency=5,
    limit_alphabet=1000, 
    wordpieces_prefix='##',
    special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]']
    )

os.mkdir('./bert-it-1')
tokenizer.save_model('./bert-it-1', 'bert-it')
tokenizer = BertTokenizer.from_pretrained('./bert-it-1/bert-it-vocab.txt', local_files_only=True)

/Users/shanmugamr/python_tests/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1930: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


In [7]:
class BERTDataset(Dataset):
    def __init__(self, data_pair, tokenizer, seq_len=64):

        self.tokenizer = tokenizer
        self.seq_len = seq_len
        self.corpus_lines = len(data_pair)
        self.lines = data_pair

    def __len__(self):
        return self.corpus_lines

    def __getitem__(self, item):

        # Step 1: get random sentence pair, either negative or positive (saved as is_next_label)
        t1, t2, is_next_label = self.get_sent(item)    

        # Step 2: replace random words in sentence with mask / random words
        t1_random, t1_label = self.random_word(t1)
        t2_random, t2_label = self.random_word(t2)

        # Step 3: Adding CLS and SEP tokens to the start and end of sentences
         # Adding PAD token for labels
        t1 = [self.tokenizer.vocab['[CLS]']] + t1_random + [self.tokenizer.vocab['[SEP]']]

        t2 = t2_random + [self.tokenizer.vocab['[SEP]']]
        t1_label = [self.tokenizer.vocab['[PAD]']] + t1_label + [self.tokenizer.vocab['[PAD]']]

        t2_label = t2_label + [self.tokenizer.vocab['[PAD]']]

        # Step 4: combine sentence 1 and 2 as one input
        # adding PAD tokens to make the sentence same length as seq_len
        segment_label = ([1 for _ in range(len(t1))] + [2 for _ in range(len(t2))])[:self.seq_len]
        bert_input = (t1 + t2)[:self.seq_len]
        bert_label = (t1_label + t2_label)[:self.seq_len]
        padding = [self.tokenizer.vocab['[PAD]'] for _ in range(self.seq_len - len(bert_input))]
        bert_input.extend(padding), bert_label.extend(padding), segment_label.extend(padding)

        output = {"bert_input": bert_input,
                  "bert_label": bert_label,
                  "segment_label": segment_label,
                  "is_next": is_next_label}

        return {key: torch.tensor(value) for key, value in output.items()}

    def random_word(self, sentence):
        tokens = sentence.split()
        output_label = []
        output = []

        # 15% of the tokens would be replaced
        for i, token in enumerate(tokens):
            prob = random.random()

            # remove cls and sep token
            token_id = self.tokenizer(token)['input_ids'][1:-1]
            
            if prob < 0.15:
                prob /= 0.15

                # 80% chance change token to mask token
                if prob < 0.8:
                    for i in range(len(token_id)):
                        output.append(self.tokenizer.vocab['[MASK]'])

                # 10% chance change token to random token
                elif prob < 0.9:
                    for i in range(len(token_id)):
                        output.append(random.randrange(len(self.tokenizer.vocab)))

                # 10% chance change token to current token
                else:
                    output.append(token_id)

                output_label.append(token_id)

            else:
                output.append(token_id)
                for i in range(len(token_id)):
                    output_label.append(0)

        # flattening
        output = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output]))
        output_label = list(itertools.chain(*[[x] if not isinstance(x, list) else x for x in output_label]))
        assert len(output) == len(output_label)
        return output, output_label

    def get_sent(self, index):
        '''return random sentence pair'''
        t1, t2 = self.get_corpus_line(index)

        # negative or positive pair, for next sentence prediction
        if random.random() > 0.5:
            return t1, t2, 1
        else:
            return t1, self.get_random_line(), 0

    def get_corpus_line(self, item):
        '''return sentence pair'''
        return self.lines[item][0], self.lines[item][1]

    def get_random_line(self):
        '''return random single sentence'''
        return self.lines[random.randrange(len(self.lines))][1]

In [8]:
train_data = BERTDataset(
   pairs, seq_len=MAX_LEN, tokenizer=tokenizer)
train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)
print_sample = train_data[random.randrange(len(train_data))]
print(print_sample)
print(tokenizer.decode(print_sample['bert_input']))

{'bert_input': tensor([  1, 580,  17,   2,   3, 146,  15, 592,  17,   2,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0]), 'bert_label': tensor([  0,   0,   0,   0, 419,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0]), 'segment_label': tensor([1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'is_next': t

In [9]:
train_data[0]

{'bert_input': tensor([    1,   230,   184,   432,   208,  1712,    34,   529,   119, 13336,
             3,     3,     3,     3, 12824,   234,  1135,   160,  6991,  4996,
         19495,   313,  1993,  1023,    16,     3,   192,   150,  8110,    17,
           542,    17,     2,   303,    15,    48,   515,   184,    11,    43,
           672,   231, 15149,   295,  7732,   105,   242,    15,   270,   173,
            11,    58,   459,   231,   146,    17,     2,     0,     0,     0,
             0,     0,     0,     0]),
 'bert_label': tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
          8254, 10999,   179,  5189,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,   275,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0, 15149,   295,  7732,   105,   242,    15,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,

In [824]:
from torch import nn
class TokenTypelEmbeddings(nn.Module):
    def __init__(self, num_token_types = 3, embedding_dim = 768):
        super().__init__()
        self.token_type_embeddings = nn.Embedding(num_token_types, embedding_dim, padding_idx=0) # TokenizEr pad id

    def forward(self, input):
        return self.token_type_embeddings(input)

In [825]:
class WordEmbeddings(nn.Module):
    def __init__(self, vocab_size, embedding_dim = 768):
        super().__init__()
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

    def forward(self, input):
        return self.word_embeddings(input)

In [826]:
class PositionalEmbeddings(nn.Module):
    def __init__(self, seq_len, embedding_dim = 768):
        super().__init__()
        self.seq_len = seq_len
        positional_embedding = torch.zeros((seq_len, embedding_dim))
        positional_embedding.require_grad = False
        
        for pos in range(seq_len):
            for i in range(0, embedding_dim, 2):
                positional_embedding[pos, i] = math.sin(pos / (10000 ** ((2 * i)/embedding_dim)))
                positional_embedding[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/embedding_dim)))
            
        self.positional_embeddings = positional_embedding.unsqueeze(0)

    def forward(self, input):
        return self.positional_embeddings
        

In [827]:
class BertEmbeddingLayer(nn.Module):
    
    def __init__(self, vocab_size, seq_len = 64, embedding_dim = 768, num_token_types = 3, dropout=0.1):
        super().__init__()
        self.positional_embeddings = PositionalEmbeddings(seq_len, embedding_dim)
        self.token_type_embeddings = TokenTypelEmbeddings(num_token_types, embedding_dim)
        self.word_embeddings = WordEmbeddings(vocab_size, embedding_dim)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, input_tokens, input_token_types):
        x = self.positional_embeddings(input_tokens) + self.word_embeddings(input_tokens) + self.token_type_embeddings(input_token_types)
        return self.dropout(x)
        

In [828]:
class Projection(nn.Module):
    def __init__(self, embedding_dim = 768):
        super().__init__()
        self.queries = nn.Linear(embedding_dim, embedding_dim)
        self.keys = nn.Linear(embedding_dim, embedding_dim)
        self.values = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, input_embeddings):
        return self.queries(input_embeddings), self.keys(input_embeddings), self.values(input_embeddings)

In [829]:
import math
class Attention(nn.Module):
    def __init__(self, num_attention_heads = 8, embedding_dim = 768, dropout=0.1):
        super().__init__()
        self.num_attention_heads = num_attention_heads
        self.dropout = torch.nn.Dropout(dropout)
        self.output_linear = nn.Linear(embedding_dim, embedding_dim)

    def forward(self, q, k, v, padding_mask):
        bs, seq_len, embed_dim = q.shape
        num_elem_per_attn_head = embed_dim//self.num_attention_heads

        qr = q.view(bs, seq_len, self.num_attention_heads, -1).permute(0,2,1,3)
        vr = v.view(bs, seq_len, self.num_attention_heads, -1).permute(0,2,1,3)
        kr = k.view(bs, seq_len, self.num_attention_heads, -1).permute(0,2,1,3)

        attn_scores = torch.matmul(qr, kr.permute(0,1,3,2))/math.sqrt(num_elem_per_attn_head)
        #ATTN SCORES SHAPE torch.Size([16, 12, 64, 64]) , PADDING MASK SHAPE torch.Size([16, 1, 64, 64])
        attn_scores_masked = attn_scores.masked_fill(padding_mask == 0, -1e9)
        attn_scores_normalized = torch.nn.functional.softmax(attn_scores_masked, dim=-1)
        attn_scores_regularized = self.dropout(attn_scores_normalized)
                        
        attn_output = torch.matmul(attn_scores_regularized, vr).permute(0,2,1,3).contiguous().view(bs, seq_len, -1)
        return self.output_linear(attn_output)

In [830]:
class FullyConnected(nn.Module):
    def __init__(self, embedding_dim = 768, expansion_factor = 4, dropout = 0.1):
        super().__init__()
        intermediate_dim = embedding_dim * expansion_factor 
        self.fc1 = nn.Linear(embedding_dim, intermediate_dim)
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(intermediate_dim, embedding_dim)
        self.activation = nn.GELU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.activation(x)
        return x

In [831]:
class TransformerLayer(nn.Module):
    def __init__(self, embedding_dim = 768, num_attention_heads = 8, expansion_factor = 4, dropout = 0.1):
        super().__init__()
        self.projection = Projection(embedding_dim)
        self.attention = Attention(num_attention_heads, embedding_dim)
        self.layernorm1 = nn.LayerNorm(embedding_dim)
        self.fully_connected = FullyConnected(embedding_dim, expansion_factor)
        self.layernorm2 = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, padding_mask):
        queries, keys, values = self.projection(x)
        attention_ouptut = self.attention(queries, keys, values, padding_mask)
        attention_output_regularized = self.dropout(attention_ouptut)
        residual_1 = x + attention_output_regularized
        layer_norm_1 = self.layernorm1(residual_1)
        fully_connected_output = self.fully_connected(layer_norm_1)
        fully_connected_output_regularized = self.dropout(fully_connected_output)
        residual_2 = layer_norm_1 + fully_connected_output_regularized
        final_output = self.layernorm2(residual_2)
        return final_output

In [862]:
class TransformerBlock(nn.Module):
    def __init__(self, num_layers = 12, embedding_dim = 768, num_attention_heads = 12, expansion_factor = 4):
        super().__init__()
        self.layers = nn.ModuleList([
            TransformerLayer(embedding_dim, num_attention_heads, expansion_factor) for _ in range(num_layers)
                       ])
        
    def forward(self, x, padding_mask):
        for l in range(1, num_layers):
            x = self.layers[l](x, padding_mask)
        return x

In [863]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, seq_len = 64, num_token_types = 3, num_layers = 12, embedding_dim = 768, num_attention_heads = 12, expansion_factor = 4):
        super().__init__()
        self.embedding_layer = BertEmbeddingLayer(vocab_size, seq_len, embedding_dim, num_token_types)
        self.transformer_blocks = TransformerBlock(num_layers, embedding_dim, num_attention_heads, expansion_factor)

    def forward(self, inp_tokens, input_token_types):
        seq_len = inp_tokens.size(1)
        padding_mask = (inp_tokens > 0).unsqueeze(1).repeat(1, seq_len, 1).unsqueeze(1)
        embeddings = self.embedding_layer(inp_tokens, input_token_types)
        encoded_representation = self.transformer_blocks(embeddings, padding_mask)
        return encoded_representation


e = Encoder(tokenizer.vocab_size)
op = e(inp['bert_input'], inp['segment_label'])

In [882]:
class NextSentancePrediction(nn.Module):
    def __init__(self, embedding_dim = 768):
        super().__init__()
        self.linear_layer = nn.Linear(embedding_dim, 2)

    def forward(self, encoded_representation, target_labels):
        cls_token_representation = encoded_representation[:,0,:]
        logits = self.linear_layer(cls_token_representation)
        nsp_loss = torch.nn.CrossEntropyLoss()(logits, target_labels)
        return nsp_loss

class MaskedLanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim = 768):
        super().__init__()
        self.linear_layer = nn.Linear(embedding_dim, vocab_size)

    def forward(self, encoded_representation, target_labels):
        bs, seq_len , embedding_dim = encoded_representation.shape
        target_labels_flattened = target_labels.view(bs*seq_len)
        logits = self.linear_layer(encoded_representation)
        logits_flattenend = logits.view((bs*seq_len, -1))
        all_loss = nn.CrossEntropyLoss(ignore_index=0)(logits_flattenend, target_labels_flattened) #reduction='none', 
        #target_labels_flattened[target_labels_flattened > 0] = 1         
        #mlm_loss = torch.dot(target_labels_flattened.float(), all_loss) / torch.sum(target_labels_flattened)
        return all_loss #mlm_loss
        
        

In [883]:
vocab_size = 200
seq_len = 64
num_token_types = 3
num_layers = 12
embedding_dim = 768
num_attention_heads = 8
expansion_factor = 4
vocab_size = tokenizer.vocab_size
batch_size = 16


from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
train_data_iterator  = iter(train_dataloader)

inp = next(train_data_iterator)


class BertModel(nn.Module):
    def __init__(self, vocab_size, seq_len = 64, num_token_types=3, num_layers=12, embedding_dim = 768, num_attention_heads = 12, expansion_factor=4):
        super().__init__()
        self.encoder = Encoder(vocab_size, seq_len, num_token_types, num_layers, embedding_dim, num_attention_heads, expansion_factor)
        self.masked_language_model = MaskedLanguageModel(vocab_size, embedding_dim) 
        self.next_sentance_prediction = NextSentancePrediction(embedding_dim)

    def forward(self, input_tokens, input_token_types, target_mlm_labels, target_nsp_labels):
        encoded_representation = self.encoder(input_tokens, input_token_types)
        nsp_loss = self.next_sentance_prediction(encoded_representation, target_nsp_labels)
        mlm_loss = self.masked_language_model(encoded_representation, target_mlm_labels)
        total_loss = mlm_loss + nsp_loss
        return total_loss
        
loss = BertModel(tokenizer.vocab_size)(inp['bert_input'], inp['segment_label'], inp['bert_label'].clone(), inp['is_next'])
print(loss)

tensor(10.8747, grad_fn=<AddBackward0>)


In [899]:
macs = profile_macs( BertModel(tokenizer.vocab_size), (inp['bert_input'], inp['segment_label'], inp['bert_label'].clone(), inp['is_next']))

NameError: name 'profile_macs' is not defined

In [884]:
class ScheduledOptim():
    '''A simple wrapper class for learning rate scheduling'''

    def __init__(self, optimizer, d_model, n_warmup_steps):
        self._optimizer = optimizer
        self.n_warmup_steps = n_warmup_steps
        self.n_current_steps = 0
        self.init_lr = np.power(d_model, -0.5)

    def step_and_update_lr(self):
        "Step with the inner optimizer"
        self._update_learning_rate()
        self._optimizer.step()

    def zero_grad(self):
        "Zero out the gradients by the inner optimizer"
        self._optimizer.zero_grad()

    def _get_lr_scale(self):
        return np.min([
            np.power(self.n_current_steps, -0.5),
            np.power(self.n_warmup_steps, -1.5) * self.n_current_steps])

    def _update_learning_rate(self):
        ''' Learning rate scheduling per step '''

        self.n_current_steps += 1
        lr = self.init_lr * self._get_lr_scale()

        for param_group in self._optimizer.param_groups:
            param_group['lr'] = lr

In [887]:
from itertools import cycle
class BERTTrainer:
    def __init__(
        self, 
        model, 
        train_dataloader, 
        test_dataloader=None, 
        lr= 1e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        warmup_steps=10000,
        log_freq=10,
        device='cuda',
        embedding_dim = 768
        ):

        self.device = device
        self.model = model
        self.train_data = train_dataloader
        self.test_data = test_dataloader

        # Setting the Adam optimizer with hyper-param
        self.optim = Adam(self.model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)
        self.optim_schedule = ScheduledOptim(
            self.optim, embedding_dim, n_warmup_steps=warmup_steps
            )

        # Using Negative Log Likelihood Loss function for predicting the masked_token
        self.criterion = torch.nn.NLLLoss(ignore_index=0)
        self.log_freq = log_freq
        print("Total Parameters:", sum([p.nelement() for p in self.model.parameters()]))
    
    def train(self, epoch):
        self.iteration(epoch, self.train_data)

    def test(self, epoch):
        self.iteration(epoch, self.test_data, train=False)

    def iteration(self, epoch, data_loader, train=True):
        
        avg_loss = 0.0
        total_correct = 0
        total_element = 0
        
        mode = "train" if train else "test"

        # progress bar
        data_iter = tqdm.tqdm(
            enumerate(data_loader),
            desc="EP_%s:%d" % (mode, epoch),
            total=len(data_loader),
            bar_format="{l_bar}{r_bar}"
        )

        for i, data in data_iter:

            # 0. batch_data will be sent into the device(GPU or cpu)
            data = {key: value.to(self.device) for key, value in data.items()}

            loss = self.model(data['bert_input'], data['segment_label'], data['bert_label'].clone(), data['is_next'])

            # 3. backward and optimization only in train
            if train:
                self.optim_schedule.zero_grad()
                loss.backward()
                self.optim_schedule.step_and_update_lr()

            # next sentence prediction accuracy
           #correct = next_sent_output.argmax(dim=-1).eq(data["is_next"]).sum().item()
            #avg_loss += loss.item()
            #total_correct += correct
            #total_element += data["is_next"].nelement()

            post_fix = {
                "epoch": epoch,
                "iter": i,
              #  "avg_loss": avg_loss / (i + 1),
              #  "avg_acc": total_correct / total_element * 100,
                "loss": loss.item()
            }

            if i % self.log_freq == 0:
                data_iter.write(str(post_fix))
        print(f"EP{epoch}, {mode}: \
            avg_loss={avg_loss / len(data_iter)}, \
            total_acc={total_correct * 100.0 / total_element}"
        ) 

In [888]:
'''test run'''

train_data = BERTDataset(
   pairs, seq_len=MAX_LEN, tokenizer=tokenizer)

train_loader = DataLoader(
   train_data, batch_size=32, shuffle=True, pin_memory=True)

bert_model =  BertModel(tokenizer.vocab_size)

bert_trainer = BERTTrainer(bert_model, train_loader, device='cpu')
epochs = 20

for epoch in range(epochs):
  bert_trainer.train(epoch)

Total Parameters: 117581226


EP_train:0:   0%|| 1/6926 [00:03<6:24:37,  3.33s/it]

{'epoch': 0, 'iter': 0, 'loss': 10.980493545532227}


EP_train:0:   0%|| 11/6926 [00:30<5:12:06,  2.71s/it]

{'epoch': 0, 'iter': 10, 'loss': 10.817377090454102}


...

{'epoch': 0, 'iter': 6910, 'loss': 6.621090888977051}


EP_train:0: 100%|| 6921/6926 [7:32:29<00:21,  4.33s/it]

{'epoch': 0, 'iter': 6920, 'loss': 6.206551551818848}


EP_train:0: 100%|| 6926/6926 [7:32:48<00:00,  3.92s/it]


ZeroDivisionError: float division by zero

In [889]:
class Counter:

    def __init__(self, start: int = 0) -> None:
        self.counter = start

    def __next__(self) -> int:
        i = self.counter
        self.counter += 1
        return i

    def reset(self) -> None:
        self.counter = 0

In [890]:
c = Counter()

In [891]:
next(c)

0

In [892]:
next(c)

1

In [893]:
next(c)

2

In [894]:
reset(c)

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Don't know how to reset  (c), please run `%reset?` for details


In [895]:
c


In [896]:
next(c)

3

In [897]:
c.reset()

In [898]:
next(c)

0